# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

Expected output:

>
>|    |   InvoiceNo |   StockCode |   year |   month |   day |   hour | Description                     |   Quantity | InvoiceDate         |   UnitPrice |   CustomerID | Country        |   amount_spent |
|---:|------------:|------------:|-------:|--------:|------:|-------:|:--------------------------------|-----------:|:--------------------|------------:|-------------:|:---------------|---------------:|
|  0 |      546084 |       22741 |   2011 |       3 |     3 |     11 | funky diva pen                  |         48 | 2011-03-09 11:28:00 |        0.85 |        14112 | United Kingdom |          40.8  |
|  1 |      545906 |       22557 |   2011 |       3 |     2 |      9 | plasters in tin vintage paisley |         12 | 2011-03-08 09:23:00 |        1.65 |        15764 | United Kingdom |          19.8  |
|  2 |      539475 |       22176 |   2010 |      12 |     7 |     14 | blue owl soft toy               |          1 | 2010-12-19 14:41:00 |        2.95 |        16686 | United Kingdom |           2.95 |
|  3 |      572562 |       21889 |   2011 |      10 |     2 |      9 | wooden box of dominoes          |         12 | 2011-10-25 09:07:00 |        1.25 |        13481 | United Kingdom |          15    |
|  4 |      549372 |       72741 |   2011 |       4 |     5 |     11 | grand chocolatecandle           |          9 | 2011-04-08 11:28:00 |        1.45 |        14958 | United Kingdom |          13.05 |

In [2]:
# your code here

# Load the dataset into a Pandas DataFrame
orders = pd.read_csv('../data/orders_sample.csv')

# Check dataset information
print(f'This dataset has {orders.shape[0]} rows and {orders.shape[1]} columns.\n')
print(orders.info())
print(f'\nLooking at the information, we can see that there is no missing values.')

# Check the dataset
orders.head()

This dataset has 20000 rows and 13 columns.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   InvoiceNo     20000 non-null  int64  
 1   StockCode     20000 non-null  object 
 2   year          20000 non-null  int64  
 3   month         20000 non-null  int64  
 4   day           20000 non-null  int64  
 5   hour          20000 non-null  int64  
 6   Description   20000 non-null  object 
 7   Quantity      20000 non-null  int64  
 8   InvoiceDate   20000 non-null  object 
 9   UnitPrice     20000 non-null  float64
 10  CustomerID    20000 non-null  int64  
 11  Country       20000 non-null  object 
 12  amount_spent  20000 non-null  float64
dtypes: float64(2), int64(7), object(4)
memory usage: 2.0+ MB
None

Looking at the information, we can see that there is no missing values.


,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,546084,22741,2011,3,3,11,funky diva pen,48,2011-03-09 11:28:00,0.85,14112,United Kingdom,40.80
1,545906,22557,2011,3,2,9,plasters in tin vintage paisley,12,2011-03-08 09:23:00,1.65,15764,United Kingdom,19.80
2,539475,22176,2010,12,7,14,blue owl soft toy,1,2010-12-19 14:41:00,2.95,16686,United Kingdom,2.95
3,572562,21889,2011,10,2,9,wooden box of dominoes,12,2011-10-25 09:07:00,1.25,13481,United Kingdom,15.00
4,549372,72741,2011,4,5,11,grand chocolatecandle,9,2011-04-08 11:28:00,1.45,14958,United Kingdom,13.05


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here

'''
For the analysis purpose, only the columns 'CustomerID', 'Country' and 'amount_spent' will be necessary. The 'CustomerID'
will be necessary to identify each customer and the 'Country' to identify where the purchase was made. The other columns,
except from the 'amout_spent', are just information about the purchase made by each client. Since what we wanto to know is
how much each customer has spent, they are not necessary, only needing the column 'amount_spent' to find that.
'''

# Check number of unique customers
'''
The customers are represented by an ID, which appears in the column "CustomerID". So, to check the number of unique
customers, it is necessary to check the unique IDs.
'''
list_unique_customerid = list(orders.CustomerID.unique())
print(f'There are {len(list_unique_customerid)} unique customer IDs.')

There are 3326 unique customer IDs.


In [4]:
# Aggregation of the 'amount_spent' for unique customers by 'Country'
'''
Since we want to information about the aggregated expenses by each customer at the global chain sotres, all the data can
be grouped by customer ID. 
'''

# Aggregate the 'amount_spent' by 'CustomerID', select only the 'amout_spent' column, reset the index and store the 
# result in a variable
orders_spent = orders.groupby(by='CustomerID').sum().loc[:, 'amount_spent'].reset_index()

# Check information about the new dataset
print(f'This dataset has {orders_spent.shape[0]} rows and {orders_spent.shape[1]} columns.')
print(f'The number of rows in the new dataframe matches the number of unique customer IDs, meaning that all customers',
      f'were considered.\n', sep=' ')
print(orders_spent.info())

# Check the result
orders_spent.head()

This dataset has 3326 rows and 2 columns.
The number of rows in the new dataframe matches the number of unique customer IDs, meaning that all customers were considered.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3326 entries, 0 to 3325
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CustomerID    3326 non-null   int64  
 1   amount_spent  3326 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 52.1 KB
None


,CustomerID,amount_spent
0,12347,149.90
1,12348,75.36
2,12349,100.09
3,12350,10.20
4,12352,126.48


In [5]:
# Create a subset of the dataframe 'orders' containg only the customer ID and the country
customer_country = orders.loc[:, ['CustomerID', 'Country']].drop_duplicates().sort_values(by=['CustomerID'])

# Check informatio about the 'customer_country' dataframe
print(f'This dataset has {customer_country.shape[0]} rows and {customer_country.shape[1]} columns.')
print(f'The number of rows in this dataframe does not match the number of unique customer IDs, meaning that some',
      f'customers have made purchases in more than one location. Since the difference is equal to '
      f'{customer_country.shape[0] - len(list_unique_customerid)}, thare are probably '
      f'{customer_country.shape[0] - len(list_unique_customerid)} of this customers.\n', sep=' ')

# Check the customers that have made purchases in more than one country
# Store in a variable the customer IDs that are duplicated
mask = customer_country.CustomerID.duplicated(keep=False)
print('The table below list the customers that have made purchases in more than one country.')
customer_country[mask]

This dataset has 3331 rows and 2 columns.
The number of rows in this dataframe does not match the number of unique customer IDs, meaning that some customers have made purchases in more than one location. Since the difference is equal to 5, thare are probably 5 of this customers.

The table below list the customers that have made purchases in more than one country.


,CustomerID,Country
1730,12417,Belgium
3446,12417,Spain
5789,12422,Australia
212,12422,Switzerland
2173,12429,Denmark
1261,12429,Austria
12412,12431,Belgium
2321,12431,Australia
8206,12455,Cyprus
137,12455,Spain


In [6]:
# Merge the 'customer_country' dataframe to the 'orders_spent'
orders_customers = orders_spent.merge(customer_country).sort_values(by='CustomerID')

# Check the result
print(f'This new dataframe has {orders_customers.shape[0]} rows and {orders_customers.shape[1]} columns.')
orders_customers.head()

This new dataframe has 3331 rows and 3 columns.


,CustomerID,amount_spent,Country
0,12347,149.90,Iceland
1,12348,75.36,Finland
2,12349,100.09,Italy
3,12350,10.20,Norway
4,12352,126.48,Norway


In [7]:
# Classify the customers
'''
Before classifying the customers, some conditons have to be prepared.
'''

# 95th quantile
q95 = orders_customers.amount_spent.quantile(q=0.95)
print(f'The 95th quantile is {q95:.2f}.')

# 75th quantile
q75 = orders_customers.amount_spent.quantile(q=0.75)
print(f'The 75th quantile is {q75:.2f}.')

# Classification
orders_customers['Classification'] = np.where(orders_customers['amount_spent'] > q95, 'VIP',
                                             np.where(orders_customers['amount_spent'] >= q75, 'Preferred',
                                                     'Regular'))

# Check the result
orders_customers.head()

The 95th quantile is 385.44.
The 75th quantile is 112.49.


,CustomerID,amount_spent,Country,Classification
0,12347,149.90,Iceland,Preferred
1,12348,75.36,Finland,Regular
2,12349,100.09,Italy,Regular
3,12350,10.20,Norway,Regular
4,12352,126.48,Norway,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [8]:
# your code here

# Number of VIPs around the world
# Select only the customers that has a VIP classification, select the column 'CustomerID' and count how many VIP customers
# there are
vip_number = orders_customers[orders_customers.Classification == 'VIP'].CustomerID.count()
print(f'There are {vip_number} VIP customers around the world.')

# Number of VIP customers by country
# Select only the customers that has a VIP classification, group by 'Country', ount how many VIP customers there are by
# country, sort the values in a descending way and reset the index
vip_country = orders_customers[orders_customers.Classification == 'VIP'].groupby(by='Country').agg(VIP = ('Classification', 'count')).sort_values(by='VIP', ascending=False).reset_index()
print(f'The country that has the most VIP customers is {vip_country.iloc[0, 0]}.')

# Check the result table
vip_country

There are 167 VIP customers around the world.
The country that has the most VIP customers is United Kingdom.


,Country,VIP
0,United Kingdom,138
1,Germany,7
2,France,6
3,Switzerland,3
4,EIRE,2
5,Norway,2
6,Spain,2
7,Australia,1
8,Belgium,1
9,Israel,1


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [9]:
# your code here
# Number of Preferred customers around the world
# Select only the customers that has a Preferred classification, select the column 'CustomerID' and count how many
# Preferred customers there are
pref_number = orders_customers[(orders_customers.Classification == 'Preferred')].CustomerID.count()
print(f'There are {pref_number} Preferred customers around the world.')

# Number of VIP and Preferred customers together around the world
vip_pref_number = vip_number + pref_number
print(f'And together with the VIP customers, there are {vip_pref_number} in total around the world.')

# Number of VIP and Preferred customers by country

# Condition to select VIP custormers or Preferred customers
mask = (orders_customers.Classification == 'VIP') | (orders_customers.Classification == 'Preferred')

# Select only the customers that has a VIP or Preferred classification, group by 'Country', ount how many VIP customers 
# there are by country, sort the values in a descending way and reset the index
vip_pref_country = orders_customers[mask].groupby(by='Country').agg(VIP_Preferred = ('Classification', 'count')).sort_values(by='VIP_Preferred', ascending=False).reset_index()
print(f'The country that has the most VIP and Preferred customers is {vip_pref_country.iloc[0, 0]}.')

# Check the result table
vip_pref_country

There are 666 Preferred customers around the world.
And together with the VIP customers, there are 833 in total around the world.
The country that has the most VIP and Preferred customers is United Kingdom.


,Country,VIP_Preferred
0,United Kingdom,722
1,Germany,32
2,France,20
3,Belgium,8
4,Switzerland,7
5,Spain,5
6,Norway,5
7,EIRE,3
8,Finland,3
9,Austria,2
